# 형태소 분석기 실험

In [1]:
import pandas as pd
from konlpy import init_jvm
from konlpy.tag import Mecab, Komoran,Kkma,Hannanum
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
csv_path = './news_crawler/news_data.csv'
csv_path2 = './news_crawler/news_data2.csv'

In [2]:
df1 = pd.read_table(csv_path, sep=',')
df1.head()

,news,code
0,파주시청. 사진제공=파주시 파주시청. 사진제공=파주시\n\n[파주=파이낸셜뉴스 강근...,사회
1,동영상 뉴스\n\n이천 물류창고 화재 발화지점으로 지목된 지하 2층에서 산소절단기의...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검. 사진...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 20대 남성이 구속됐습니다.서울 강남경찰서는 ...,사회
4,(서울=연합뉴스) 대한약사회가 6일부터 코로나바이러스 감염증 대응 체계를 '사회적 ...,사회


In [3]:
idx2word = {'101' : '경제', '102' : '사회', '103' : '생활/문화', '105' : 'IT/과학'}
labels = ['경제', '사회', '생활/문화','IT/과학']
stopwords = ['에','는','은','을','했','에게',
             '있','이','의','하','한','다',
             '과','때문','할','수','무단',
             '따른','및','금지','전재',
             '경향신문','기자','는데','가',
             '등','들','파이낸셜','저작','등','뉴스']

In [4]:
df=df1
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

0       파주시청 사진제공파주시 파주시청 사진제공파주시파주파이낸셜뉴스 강근주 기자 파주시는 ...
1       동영상 뉴스이천 물류창고 화재 발화지점으로 지목된 지하 층에서 산소절단기의 산소 공...
2       황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검 사진제...
3       귀갓길 여성을 쫓아가 성범죄를 시도한 대 남성이 구속됐습니다서울 강남경찰서는 강간상...
4       서울연합뉴스 대한약사회가 일부터 코로나바이러스 감염증 대응 체계를 사회적 거리두기에...
                              ...                        
5119    신종 코로나바이러스 감염증코로나 사태 이후 가정의 달 월에도 언택트비대면 신풍속도가...
5120    는 소비자로부터 월 이용료 만만원을 받고 초고속 인터넷을 제공한다 그런 브로드밴드가...
5121    머리를 긁고 있는 오랑우탄 몸을 긁는 행동을 따라 하는 것은 부정적 감정과 관련이 ...
5122    가 오는 일 정식 출시하는 스마트폰 벨벳이 사실상 공짜폰이 될 전망이다 단말기 가격...
5123    이미지제공게티이미지뱅크 이미지제공게티이미지뱅크  전자신문  전자신문인터넷 무단전재 ...
Name: news, Length: 5124, dtype: object

In [5]:
# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수입니다.
def preprocessing(data,mode,stopwords):
  text_data = []
  mode = mode.lower()

  if mode == "mecab":
      tokenizer = Mecab()
  elif mode == 'komoran':
      tokenizer = Komoran()
  elif mode == 'kkma':
      tokenizer = Kkma()
  elif mode == 'hannanum':
      tokenizer = Hannanum()        

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = tokenizer.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [6]:
def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

In [7]:
import time
start=time.time()
text_data1 = preprocessing(df['news'],"Mecab",stopwords)
print("Mecab 측정된 시간 : ",time.time()-start)

start=time.time()
text_data2 = preprocessing(df['news'],"komoran",stopwords)
print("komoran 측정된 시간 : ",time.time()-start)

start=time.time()
text_data3 = preprocessing(df['news'],"kkma",stopwords)
print("kkma 측정된 시간 : ",time.time()-start)

start=time.time()
text_data4 = preprocessing(df['news'],"hannanum",stopwords)
print("hannanum 측정된 시간 : ",time.time()-start)

Mecab 측정된 시간 :  6.528712749481201
komoran 측정된 시간 :  59.331480264663696
kkma 측정된 시간 :  986.476019859314
hannanum 측정된 시간 :  138.00313234329224


In [8]:
#Mecab
X_train, X_test, y_train, y_test = train_test_split(text_data1, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

# komoran
X_train, X_test, y_train, y_test = train_test_split(text_data2, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

# kkma
X_train, X_test, y_train, y_test = train_test_split(text_data3, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

# hannanum
X_train, X_test, y_train, y_test = train_test_split(text_data4, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.92      0.85      0.88       355
          사회       0.79      0.91      0.85       500
       생활/문화       0.87      0.78      0.83       426

    accuracy                           0.85      1281
   macro avg       0.86      0.85      0.85      1281
weighted avg       0.86      0.85      0.85      1281

              precision    recall  f1-score   support

       IT/과학       0.92      0.85      0.89       355
          사회       0.80      0.89      0.84       500
       생활/문화       0.84      0.79      0.81       426

    accuracy                           0.84      1281
   macro avg       0.85      0.84      0.85      1281
weighted avg       0.85      0.84      0.84      1281

              precision    recall  f1-score   support

       IT/과학       0.92      0.85      0.88       355
          사회       0.80      0.90      0.85       500
       생활/문화       0.86      0.79      0.82       426

    accuracy        

현재 성능은 비슷한데 kkma와 hannanum은 속도가 너무 느려서 사용하기 힘들다.

# 불용어 추가

In [9]:
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스','노컷뉴스','매일신문','그','나오','가지','씨','시키','만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신','안','어떤','내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번','나','다른','어떻','여자','개','전','들','사실','이렇','점','싶','말','정도','좀','원','잘','통하','소리','놓','권','자','재','될','고','되','며','주','인','시','일','파','만','총','게','태','스','크','를','어','당','각','지난','관련','지만','사람','입니다','습니다','밝혔','지만','면서','지난','우리','이상','이용','경우','뉴시스','까지','다는','다면']

In [10]:
text_data1 = preprocessing(df['news'],"Mecab",stopwords)
text_data2 = preprocessing(df['news'],"komoran",stopwords)

#Mecab
X_train, X_test, y_train, y_test = train_test_split(text_data1, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

# komoran
X_train, X_test, y_train, y_test = train_test_split(text_data2, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       IT/과학       0.92      0.85      0.88       355
          사회       0.79      0.90      0.84       500
       생활/문화       0.86      0.77      0.82       426

    accuracy                           0.84      1281
   macro avg       0.86      0.84      0.85      1281
weighted avg       0.85      0.84      0.84      1281

              precision    recall  f1-score   support

       IT/과학       0.92      0.84      0.88       355
          사회       0.79      0.88      0.84       500
       생활/문화       0.83      0.78      0.81       426

    accuracy                           0.84      1281
   macro avg       0.85      0.83      0.84      1281
weighted avg       0.84      0.84      0.84      1281



불용어를 추가했는데 왜인지 정확도가 낮아졌다.....

# 데이터 합치기

In [11]:
df2 = pd.read_table(csv_path2, sep=',')
df2.head()

,news,code
0,기사 섹션 분류 안내\n\n기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다...,경제
1,▶제21대 총선 실시간 개표 현황 및 결과 보기\n\n총선에서 여당이 다시 한 번 ...,경제
2,[뉴욕=AP/뉴시스]지난 10일 뉴욕 증권거래소 건물에 미국 국기가 게양되어 있다....,경제
3,"부산지역 주유소에서 판매하는 기름값이 휘발유는 평균 1200원대, 경유는 1000원...",경제
4,담배업계가 소비자의 취향을 저격한 다양한 담배 신제품들을 잇달아 선보이고 있다.이전...,경제


In [12]:
df=pd.concat([df1,df2],axis=0,ignore_index=True)

In [13]:
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

0       파주시청 사진제공파주시 파주시청 사진제공파주시파주파이낸셜뉴스 강근주 기자 파주시는 ...
1       동영상 뉴스이천 물류창고 화재 발화지점으로 지목된 지하 층에서 산소절단기의 산소 공...
2       황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검 사진제...
3       귀갓길 여성을 쫓아가 성범죄를 시도한 대 남성이 구속됐습니다서울 강남경찰서는 강간상...
4       서울연합뉴스 대한약사회가 일부터 코로나바이러스 감염증 대응 체계를 사회적 거리두기에...
                              ...                        
8822    레고켐바이오사이언스대표 김용주사진가 글로벌 기술수출에 또다시 성공했다 년 중국 포순...
8823    제대 국회의원선거 기간 중단됐던 네이버 실시간 급상승 검색어 서비스가 일 오후 시부...
8824    아마존 연합뉴스 자료사진 아마존 연합뉴스 자료사진직원들 코로나 확산 방지 조치 충분...
8825    제대 국회의원선거가 임박한 가운데 투표 및 개표 방송을 준비하는 기업들의 움직임이 ...
8826    롤 점검이 일 진행될 예정이다온라인게임 리그오브레전드이하 롤은 일 오전 시부터 오전...
Name: news, Length: 8827, dtype: object

In [14]:
text_data1 = preprocessing(df['news'],"Mecab",stopwords)
text_data2 = preprocessing(df['news'],"komoran",stopwords)

#Mecab
X_train, X_test, y_train, y_test = train_test_split(text_data1, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

# komoran
X_train, X_test, y_train, y_test = train_test_split(text_data2, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       IT/과학       0.76      0.90      0.83       506
          경제       0.85      0.33      0.48       247
          사회       0.79      0.91      0.84       760
       생활/문화       0.85      0.79      0.82       694

    accuracy                           0.80      2207
   macro avg       0.81      0.73      0.74      2207
weighted avg       0.81      0.80      0.79      2207

              precision    recall  f1-score   support

       IT/과학       0.76      0.88      0.82       506
          경제       0.86      0.34      0.49       247
          사회       0.78      0.90      0.84       760
       생활/문화       0.85      0.78      0.81       694

    accuracy                           0.80      2207
   macro avg       0.81      0.73      0.74      2207
weighted avg       0.81      0.80      0.79      2207



현재 데이터에는 아래와 같이 code가 다름에도 중복되는 기사내용이 있다.\
이 부분을 덜어내고 각 code 별로 데이터가 불균등 하기 때문에 이를 가장 적은 수로 맞춰 준후에 카테고리가 다름에도 중복되는 단어들을 불용어로 사용한다.

In [15]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
8822     True
8823     True
8824     True
8825     True
8826     True
Length: 8827, dtype: bool

In [16]:
df.describe()

,news,code
count,8827,8827
unique,6127,4
top,기사 섹션 분류 안내기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다 언론사...,사회
freq,861,3000


In [17]:
df=df.drop_duplicates(['news'])
df.describe()

,news,code
count,6127,6127
unique,6127,4
top,앵커국내 이주 아동 중 상당수는 한국에서 태어나 자라왔지만 체류자격이 없는 미등록 ...,사회
freq,1,2222


In [18]:
len1 = len(df.loc[df['code']=='사회'])
len2 = len(df.loc[df['code']=='경제'])
len3 = len(df.loc[df['code']=='생활/문화'])
len4 = len(df.loc[df['code']=='IT/과학'])
min_len = min(len1,len2,len3,len4)

rate1 = (len1-min_len)/len1
rate2 = (len2-min_len)/len2
rate3 = (len3-min_len)/len3
rate4 = (len4-min_len)/len4

df=df.drop(df.loc[df['code']=='사회'].sample(frac=rate1).index)
df=df.drop(df.loc[df['code']=='경제'].sample(frac=rate2).index)
df=df.drop(df.loc[df['code']=='생활/문화'].sample(frac=rate3).index)
df=df.drop(df.loc[df['code']=='IT/과학'].sample(frac=rate4).index)
df

,news,code
6,코로나에 걸렸다 나은 친구는 아무래도 좀 멀리하게 될 것 같아요 재발 가능성 때문에...,사회
10,안병용 의정부시장 이천화재 희생자 합동분향소 헌화 사진제공의정부시 안병용 의정부시장...,사회
12,동영상 뉴스앵커다음 주 고등학교 학년부터 순차적인 등교수업이 시작되는데 교실 방역에...,사회
16,메건 마클 왕자비와 아들 아치의 모습세이브더칠드런 인스타그램 캡처 재판매 및 금지...,사회
18,기사 섹션 분류 안내기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다 언론사...,사회
...,...,...
8317,서버용 수요 감소 가능성 커져스마트폰 판매 급감도 영향상반기는 그럭저럭 버티겠지만 ...,IT/과학
8318,레고켐바이오사이언스대표 김용주사진가 글로벌 기술수출에 또다시 성공했다 년 중국 포순...,IT/과학
8319,제대 국회의원선거 기간 중단됐던 네이버 실시간 급상승 검색어 서비스가 일 오후 시부...,IT/과학
8320,아마존 연합뉴스 자료사진 아마존 연합뉴스 자료사진직원들 코로나 확산 방지 조치 충분...,IT/과학


In [19]:
texts=dict()
for label in labels:
    texts[label]=[]

context = df['news'].apply(lambda x: str(x).split())
for i,(words,label) in enumerate(zip(context,df['code'])):
    texts[label]+=words

    
counts=dict()
for label in labels:
    counts[label] = pd.Series(texts[label]).value_counts()
    print(label,counts[label])

경제 수         1407
등         1403
있다        1055
일         1023
코로나        971
          ... 
일방적          1
키운다는         1
비판했다개의       1
보답차원으로       1
급락세를         1
Length: 49777, dtype: int64
사회 일            1385
기자            974
등             876
수             854
및             809
             ... 
상태다충북도교육청       1
보급된다대치동         1
홍보도             1
받아왔다번째          1
밝혔습니다유진환        1
Length: 51937, dtype: int64
생활/문화 수           1576
일           1125
등           1073
및           1029
있다           938
            ... 
방위로            1
포착된다일          1
폭력적인           1
이승환            1
계획이다결함시정       1
Length: 68070, dtype: int64
IT/과학 수         2101
등         1342
및         1294
일         1169
있는        1046
          ... 
컴퓨터나         1
멈추도록         1
금액의          1
휴미라          1
직조방식으로       1
Length: 52154, dtype: int64


In [20]:
def duple_words(counts,stopwords):
    tmp_keys = list(counts.keys())
    c_keys=[]
    for k in tmp_keys:
        if len(counts[k]):
            c_keys.append(k)

    check = dict()
    duple= []
    for i,k in enumerate(c_keys):
        if i:
            for s in counts[k].keys():
                if s in check.keys() and s not in stopwords:
                    duple.append(s)
        else:
            for s in counts[k].keys():
                check[s]=1
    
    duple_dict = dict()
    for k in duple:
        duple_dict[k]=0
        print(k)
        if k in counts['사회'].keys():
            duple_dict[k]+=counts['사회'][k]
        if k in counts['IT/과학'].keys():
            duple_dict[k]+=counts['IT/과학'][k]
        if k in counts['생활/문화'].keys():
            duple_dict[k]+=counts['생활/문화'][k]

    duple_dict = sorted(duple_dict.items(),key=lambda x:-x[1])
    return duple_dict

duple_dict = duple_words(counts,stopwords)
print(duple_dict[:50])


있다
있는
년
코로나
위해
것으로
오후
후보가
대한
재배포
대
더불어민주당
함께
등을
통해
후보는
위한
미래통합당
서울
이날
후보
하고
중
국회의원
대해
더
김
무단전재
가운데
따르면
지역
하는
따라
뒤
제공
이번
현재
있다고
총선
제대
생활
있도록
받고
민주당
지난해
일부터
후
또
이후
지난달
모두
오는
혐의로
같은
두
명이
큰
후보를
것은
로
당시
도
화재
것이라고
마스크를
것
마스크
경영권
것이
관계자는
혐의를
공감언론
때문에
때
가장
했다
많은
온라인
신종
없는
이재용
사회적
삼성전자
씨는
지원
코로나바이러스
선거
뉴스코리아
한다고
광주
관련해
것을
삼성
전남
등에
최근
오전
문재인
자신의
출마한
차
않은
모든
일부
중인
않고
후보의
말했다
있다는
방역
사진
결과
등의
동안
여러분의
만큼
받은
직접
통합당
열린
승계
있다며
총선에서
등이
대구
코로나로
억
보고
과정에서
적극
추가
이를
감염증코로나
만에
후보와
많이
기준
명을
분
부산
밝혔다
등으로
부회장의
그는
있습니다
당선
충북
또는
세
보면
동영상
소중한
하지만
대상으로
현장
아니라
국내
경기
년생
있을
채
시장은
명의
거리
최대
아직
건
오늘
하지
첫
올해
그리고
이어
문제
저작권자
최종
없이
개표
경북
수사를
정부
필요한
전국
미국
학교
새로운
경기도
그런
국회
된
진행
특히
정
제보를
문을
노조
그동안
역시
다양한
회장은
사건
일까지
있어
해당
네이버
관련뉴스
여
거리두기
확진자
구독하기
있고
어려움을
판정을
결과를
메인에서
감염
울산
제대로
온
재판매
한다
있지만
최선을
정부의
앞으로
앞두고
검찰에
받는
이에
바
수사
대학
억원을
표
받아
주요
반드시
징역
거리두기가
확진자가
바로
앞서
받을
영상을
연합뉴스
대표
확실시
기간
않았다
두고
이미
어려운
측은
여전히
달
국민의
인해
거쳐
앞에서
크게
신규
얻어
거리두기로
간
년간
교수
아닌
최
재배포금지
부회장
피해
등은
년부터
경남
하루
역할을
각종
이렇게
회장
씨가
한다는
명으로
기자회견을
사업
제
원을
여러
등과
개표가
확진자는
강원
없다고

의식을
문제와
만의
강조했다그러면서
플라워
명동
사업의
규정에
나서고
기대에
그래도
중심의
희망하는
지상
생각이
열화상
역대
증가한
종
세계적인
수천만
지금도
전량
주민의
사랑하는
업체를
초반
달하는
맡고
예정이며
선정됐다고
식당을
입을
사옥
아이를
가족과
갑자기
자료
통해서
추진한다고
체계로
포부를
클러스터
으로
상품권을
대비한
하기로
반
태도
신고
쓸
강좌를
석을
상관없이
원래
하겠다는
건이
운동
있으나
대폭
아끼지
기업으로
법과
항의
인수한
이뤄질
높다
무거운
팀을
한꺼번에
챙긴
보건복지부
들어간
무너진
사랑을
만장을
월일부터
최근에는
않기
늦게
하거나
추정하고
창출
억원대
더불어민주당이
점검을
설명하고
직원들은
펼치고
이르는
대응을
기업들이
지침에
유지
세부
통화에서
고양시
완공을
아니지만
대출을
고민이
제도
타당성
시청
국내에
실시하고
아니라는
손에
이유가
구미
주제로
가격에
도입
내용으로
자회사
판단해
시작하는
마무리할
막을
신속한
기피
바라는
자녀
판결을
섹션으로
현
생각한다며
초등학생
하게
구축한
기여할
대출과
설치하고
걱정이
내놓은
기록해
활기를
파악하고
년으로
인증
사과와
홍
않다
대구시
시대를
관광객
자발적
불가피한
비춰
발생했다
원희룡
확정된
끝나지
노력이
운영이
여론이
감시
이곳에서
차례로
침체된
현황을
무엇을
우리나라
식으로
어려움이
진정한
있을까
훈련을
사업과
이르면
후에는
의무를
촉구하는
유지하는
김천시
얻는
물류센터
그러면서
중단
가능하도록
홀로
정당
옆에
따뜻한
요구에
임대료
유일하게
프로젝트
필요하다
배송
일상
단순히
지역에서는
자격은
호선
축하
개선
이해찬
법안이
아직은
쌀
층에서
증거를
본인의
승기를
황
사가
종일
지금은
강의로
행복을
정부에서
대회의실에서
위반한
총선에서는
캠페인을
입지
높게
늘어
있었다는
강화하고
데이터를
보면서
제조
의견
찾아볼
득표율은
발언이
물품
지정된
것인지
민주당은
보호를
사라진
예약
돕고
결과로
되도록
노무현
양향자
직전
관련이
달라진
대전의
신청은
오래
조치가
침해
앞당겨
주민들이
회사에
병

가입해
서울시의
마무리될
기자광주뉴스
몰려
웰컴
비판했다
청주를
확대할
그려
울산의
강조하고
신뢰와
연동
상해를
주민을
경험
빠졌다
선거에서도
재택근무를
판단했다
협의가
유명
수립
행정안전부
추세가
지나가는
나섰고
그쳤다
계획도
갈수록
기업하기
인터넷을
알려졌다일
나은
운전자를
미증유의
구매할
판매된
검사와
선택이
시점이
대유행에
강좌
낡은
진행했다며
태국
수행하는
빚을
마스크에
마스크와
일자
주가
가득한
발생하면서
완화에
업체에
수요를
어린이들이
제목의
최근에
명령과
조정을
꽃
활동에
캐나다
카드
남부
필요하고
움직임이
감염증으로
부부는
막
보내기
잃는
낮다는
주요기사로직접
미국이
여건
국가대표팀
자녀의
대화
놓인
컸던
회원과
재고
차들이
더해
나를
끝났다
가격
띠고
신호가
전년도
어디
대학원
회사에서
했다또한
고위험
취지에
늦은
제고
일자리가
처리해야
회복하는
활동과
소통으로
원해요
삼바
측면이
함으로써
기대된다
쓰이는
강의와
수준이
기관에
생각하는
기대하기는
현상을
높아지고
나가
편취한
전현직
줄고
분석도
장애인이
집을
챌린지를
카드사
편성한
성적을
상용화
선물
공감하고
공익
저학년
접수한
거리에서
언론인
정도면
멈춰
봤을
미뤄졌던
효과와
신혼부부
시너지
혁신과
권리
쓰레기를
상황이지만
식당과
제외하고는
감소세를
전염병
맞이해
여부에
대선
많습니다
연합뉴스서울경제
인프라가
공석이었던
걱정도
오르게
결과에서는
점검과
원씩
친노
연결
쏟아지면서
상품
높고
개월의
판단은
나고
지키지
실시한다고
전반에
감독
원격수업을
불편함이
의자
잠을
이유에
공개된
많지
대구와
일에도
시점을
미
떨어지는
유리하게
그래픽
추락한
로봇
외출
정도는
후속기사
차관
과정이
써
냈다
아픈
신호를
막고
막상
국민께
격전지로
수치가
전형
신사업에
실려
넘은
성숙한
이병래
끊는
기술력과
유럽연합
지시한
표시가
지적에
호재가
수준에
선로
휴식을
아닙니다
매출액
배포한
감소하는
손님들이
위해선
작성한
올바른
구성돼
공식적인
포스터
두고도
방법에
관계자들은
자칫
확실하게
창출을
회복에
마련하라고

쓰레기가
말했다또한
부문
공항에서
제터미널
필요해
부합하는
고민하던
종합하면
과정은
가동을
달성하는
천억
취업이
사업장에
지켜보면서
참여했다
육성하겠다고
가동되고
후보왼쪽와
경쟁이
입
말했다그러면서
줄어들
억을
정치권과
공공
갖춰야
우희종
사회복지공동모금회
해수부
기관을
일정은
지도자
대상을
내부를
월부터는
현명한
연중
성과
맞선
표현한
엄중하게
가동하는
이동
연합
셈이다이런
발표에
저하를
지나가고
완화할
이외의
내몰려
꾸준한
봉쇄령을
유행
화재와
전동
촉구
쉽고
핵심인
정치권에
원하지
발휘해
유지하기
들어가는
이론
역할도
생산해
빌려
지역이다
대구시와
경향이
여파가
소득세
신뢰가
총선의
화학과
보장하는
주목
교환할
측정을
이용자들은
사업자
쪽
직간접적
실적을
이어가고
주거나
인력을
중의
높았다
챙기는
정례브리핑에서
재원을
해외로
성향이
현지시간
구매에
의도가
삼고
넘으면
직관적
점수는
기대도
줘야
시즌
토종
건이나
방지와
의견에
옆
교통안전
층지상
사무실을
입법
품고
선거에선
활력
근무하고
첨단소재
기대감이
영국의
만족도를
한계를
소유의
붐
월등히
튜닝
목소리로
온라인으로만
연구용역을
받아들이지
않았기
절반을
중요하지만
세월을
배까지
활용이
대표적이다
단축
시중에
보이기
미래에셋대우
감염증코로나이
두자릿수
확보할
전망
단단한
않지만
아태
계좌를
단위
횟수는
아래서
김모씨는
악성
한다고도
무게와
투자자금
대표로
이유를
죽는
원과
압도적으로
밝혔다이들은
확보가
관리할
모색할
예정이다일
시험을
복합
위상을
상사의
주주로
해고자
여행업
전면적인
적용되지
주장이다
최소화
수출을
대표인
상생
비닐장갑
넘어서는
올랐다
흐름은
메이커
경우다
가구다
사항이
인정
환경부는
늘어나자
지원한다는
때의
기대된다고
경제가
유지해야
생산의
제반
유형
이곳에
개국에서
도심의
드린
세계에서
소외계층을
서게
공기
투표에서
가져간
시정을
미래지향적
매출액은
분식
추천을
물
공감하는
품목의
소독을
국장은
양손에
조건과
것임을
중요성이
블루
이전에는
있게끔
당국의
경계를
투표소를
공간에
문화체육

일시적인
근로자와
공공주택을
지원군이
향이
질병관리본부와
분부터
줄면서
카테고리
학원이
마트에서
기회
노출되지
내용으로는
힘쓰는
수장으로
합산
음식물이
순수
커졌다
작업도
있습니다이이슬
만건이
문건
개사는
가정한
된다이
브라질의
광주광역시청
모델과
바뀌었고
시즌이
완성차
업체다
순간에
하나투어
골라
초박빙
연봉
그쳤고
서울을
오토바이
중고자동차
차종에
아이템이
징수를
투명
불안정
거세다
급한
희망이
저울질
재벌
낮아진다는
제임스
미래차
알고리즘
배터리
지부장은
패턴을
시차를
카테고리를
임기가
사항인
비용은
가능합니다
참여하지
특보가
말에는
덕분이다
축구장
재정정책과
김양수
소득공제
장관을
낮추고
기준과
조정해
년여간
속은
출처가
비하면
유발할
확장이
넷플릭스
포스코는
운동이
만가구
시점까지
입었다
자연
한다이번
건국대
경북에서
수석부회장
기울일
제한도
부당하게
앱이
말했다박상구
보험료
사라지면서
자문
원칙은
설명했다특히
나날을
불거지자
지표
규제완화
전망이다민주당은
졸업생
동참하며
확정할
포함하는
좋게
사칭한
남다르다
예상했다
탄력이
이뤄낸
올해로
주관의
휴일
전략이
다가온
초대형
마련으로
신뢰할
전략도
경력
나라는
취합
단계부터
투어
식품의약품안전처는
비서실장
세종시는
마포구에
사라졌다
주일에
수출에
부문이
주한
만장
주목해야
승기
공약이
전망해
청년일자리
달성하고
치열해지고
커지는
활용되고
강조하며
떨어졌다는
위치해
한수원
입에서
순위는
감소폭이
요구했지만
관제
주도하는
한끼
뿌리
전염병의
시까지만
식물
중요해지면서
현지에서
명분을
신을
나홀로
내외의
각별히
하는데요
이로써
붕괴는
이전인
기업부담금
사정에
규제와
줄지는
이동할
정책이
김종호
불확실성을
먹기로
입지로
앞당긴
기업이다
고용보험의
유형을
품목을
대처
됐을
볼펜으로
월까지는
소환과
펼쳤지만
잡아야
야기할
마무리되지
경제활력
개장했다
연령대
변했고
판매하겠다는
오르고
좋았다
내놓았다
대법관는
만명에
등에서도
프로모션을
판결이
시작과
진단장비
전기를
준하는
교육사업을
뉴스일
고용보험기금
원내
포항지역

...

봉쇄가
본사에
시장경제에
해소될지
발생하기
여행절벽으로
그친
확대하면서
일대에
유럽을
어려워졌다며
압박에
유사
한세엠케이가
주택가
육아용품
구입하는
협력사가
에너지로
접어들면
우희종
저소득
예상과
입자
빈익빈
치달을
유입돼도
노선
회가
앞당기는
용어
무너질
의료기기
않다면서도
이미지에
라인에는
증자를
명이다일
모자
선호한다
종목이
환급하는
일가
여름까지
쇼크가
전반과
환경오염
회장이
코나도
전달된
건에서
직원들을
만대다
가입할
금리
지원정책을
설명하며
니즈에
활발해지는
신규확진자
개월분을
활동에도
근무
마일리지
저감에
멤버십
안정성이
유기농
관측이
후보자들의
나눠주는
현저히
크림
이슈에서
출장
책임연구원이
대규모로
벨벳의
어려웠다
제조
열성적으로
팔렸으며
냄새에
년만에
디자이너서울뉴스
엇갈리는
우려해
화제인
자녀가
바뀔
만족감을
카드로
보인다앞서
개정을
머스크
대선에서
대비가
단시간에
발매한
배터리의
하고는
영상의
업황
전격
합의하는
급감했고
모양은
거론된다
계획이다또한
문제들을
유형과
확대하겠다고
행위도
불구
차단해
진단이
빌보드가
향상시키는
늘리고
오송
예외가
전문업체
방향과
신선
진행했다고
금융당국의
전파가
시청률을
자랑한다
억원과
선명하게
대부분을
장난감
사업장은
두려움에
임대료를
공장을
현지에서
빚었던
국무장관
중대한
주택을
해주길
주간의
휘발유
최악으로
영역에
귀를
동기보다
유입과
이사로
자리나
갈렸던
베개
받았던
대학의
해왔지만
극심한
낮춘
반영하지
정점으로
다음이
조절이
유통업계
거래액은
온라인교육
우편을
신제품이
기대했지만
세종뉴스
구성부터
밀집된
도모
충분하지
도요타
찍는
근무하며
소재는
국회의
업체별로
유니클로는
저하
자체도
자릿수를
줄이려면
단비
내놓을
핑크
기자파이낸셜뉴스
제공코로나
이르지만
익산시는
이용은
플랜
유지하는데
전파한다
전문기업
운항하는
마련되는
연장과
연휴가
시나리오를
발탁돼
올라서는
꺼내
맡을
생각에는
과제를
발표된다
유수
가능해진
버리는
압도적인
효율성과
올랐으나
대전의
임상시험으로
발급받은
통제에
확충을
발표한다
잃었

조정
홍콩
들이
건으로
바꾸는
건을
접근할
시급한
자가격리
기부
요청하는
높아지는
번에
블루
대행
그룹에
발굴하고
시장에서도
손실
못할
단위의
공장과
단독
파킨슨병
네이버에서
참석해
앞당겨질
구축하고
전인
끌고
촬영을
안전한
명에게
자랑하는
아끼지
혜택
차단을
브랜드로
지갑
조건
인원이
본회의에서
관점에서
근거를
최소화하기
프로세스를
관련뉴스
통과한
시장에서의
처리할
설립한
취소되면서
사장은
심사를
불과하다
활용되고
기간보다
지역은
특허
이전에
증정한다
오래
문의가
환자의
실질적인
업체들의
수석
이번에는
총선에서
이유를
판단한
주행
경쟁력이
초기에는
설명하고
만명에
높다
출신
유지를
검색어를
에디션을
실현
있지
개학
감염된
전망이
트래픽이
뒤를
보인다고
확인된
신청한
도입과
규정을
이끌어
분쟁
버전의
위주의
말에는
기록
이통사의
균형
활용하기
전문성을
액체
조사한
요청
체결하는
있다라며
보였던
이르는
어디서나
성과가
뉴스를
의해
정확하게
이용하면
소개한다
상생
대표적
권한을
화면을
인터뷰에서
한국에
추이를
줄이기
했으나
사업도
합리적인
더하면
연평균
개표
신개념
최적화
할인된
들여
만화
오류가
있으면
이뤄졌다
전액
시리즈가
월별
시작했다고
부분은
그렇지
관리를
설명하는
중국과
문제다
연
수술
신종코로나바이러스
안전을
현재는
그만큼
지난해보다
지식을
받기
제고
키워드를
강조했다이어
성장한
때보다
지식
협업을
충격을
모델들이
대해선
한국과
맞게
불확실성이
때까지
조정이
철
마련했다고
다르게
보조
성공을
신속한
배경을
사라진
선정돼
투표소에서
뉴스가
선착순
시행
간에
정상
군은
긍정적인
비용은
기반한
통찰력을
관심
회장은
전략의
수급에
봉쇄되면서
흐름을
그렇게
체질을
꼽힌다
즐겁게
늘었다고
어려워진
차례
편집
잇달아
확대되고
트레이닝
개선해
생기는
매장에서
재료를
소통과
이처럼
단계에
안내
시행을
기능의
추세가
꾸준한
선택을
법이
메시지는
자산
듯한
가족과
진행했다고
당장
활용에
작업이
비율이
작품
행사에서
소재를
취하도록
없었다
이미지에
기대가
보다는
가로로
기타

출시하는
부진했다
클래스
보여줬다고
변수로
다시금
충분하지
빅데이터를
현실적이고
아쉬움을
보안성을
사유
개에
웹사이트에
차지한
가동률
바뀔
밀집된
제주도에
중국산
대표의
조달
만이다
어려움에
선호도가
의지도
두면
감지하는
발표가
나서
대까지
신청하면
접근성은
내비게이션
카카오톡을
결정되는
주소를
온오프라인
대는
자산인
신형
차질을
이상은
있다미국
상태지만
예상된다면서
위치에
일본에
구체적
요청했다
국제고등교육혁신센터와
애플리케이션에
후에도
신제품이
화웨이샤오미
원이다
평가다일
영업을
발표는
데이비드
이유에서다
적고
했다한편
온라인에
면역반응
두번째로
정육
대중
폴더블폰에
이번엔
타입으로
주를
아래에
대상으로는
높았고
타격
당국
반대하고
콘테스트
것인지
운영사인
제휴점과의
예상됐다
쌍방향
수준까지
아프리카에
교육정보기술연구소
위는
수집해
민생과
민주당과
인수와
유권자
살균
연관
대응한
했다또
클릭해
규제는
거뒀다
제외
입는
머물렀다
업체들에
때문이다특히
절차에
아성에
달할
살려
플레이스테이션
밝혔다신종
대하여
제일
위탁
오명을
달성해
지수
비싸고
자녀를
규제하는
강화하겠다고
후보로
거는
입장은
배경에는
드문
나서야
자연
투자의
감소했지만
녹아
보완한
부터
어디에나
맛을
위축되고
극히
지속적
병의원
희망한다고
육박하는
중앙재난안전대책본부
초기부터
정해져
노트북을
사라지면서
틀을
권리를
지출
않더라도
지급이
담당자
아무것도
지원한다또
시작되면서
상반기에
씨와
피하기
기자간담회에서
독과점
부상
금융회사
에이스
사라지면
채무
억원대로
나눈
불만을
설정해야만
않지만
부족으로
이외에
이뤄낸
행보가
자제하고
취지다
구성한
패널
배치를
개념을
적자
대상이다
케미칼
고민이
당기순이익
밝혔다년
제조업체가
백신의
현대차
분들이
쓰일
모두를
그치지
전제로
쪽으로
원까지
향후에도
밸리의
관련업계에
빠졌다
집을
미치지
계속될
간담회에
초광대역폭을
안전성이
용도를
억달러를
먹고
기획전은
표시된
순조롭게
투입되며
개정해
진단했다
국가별로
능력과
잔액은
자극하는
층에
규정
제공을
않는다면
미

낮으면
현상도
인상을
미래차
분사를
남짓한
어린이가
생산시설을
응원합니다
불완전판매
의무적으로
채용공고에서
매진하고
중소기업계는
꼽았지만
이견
올랐고
제안
출처가
초기에
문서에
수집된
방법이다
자리잡고
의사소통을
이름에서
생각하지
매력적으로
중국에서는
주관
반대하는
이고
불안한
조화로운
협의에
안팎이
자극할
제휴점과
다음날
밟을
프랑스의
목전에
저작권
판매율을
표출되지
추천했다
제공지난달
지적한
우한
장갑을
주겠다고
웨비나도
만족도를
제동
마련하기로
급감
배출
있었다며
전환하고
객관적인
응시자격과
여섯
취하지
프로그램과
응답자는
다양성
경쟁입찰방식으로
보안과
수준과
와이파이의
블로그나
연령별
경제활성화
일었던
대선
상파울루
살아남을
확산했다
예측하는
넓혔다
저마다
줄어들며
명도
사안에
다룰
전파가
보건가족복지부에
아무런
지속한다는
매몰되지
길거리에서
변형이
텍사스산
정확도가
흑자
품목으로는
상승으로
있는데요
양상이
정리
걸린다며
전기차의
입국을
민첩한
따른다
정상화할
제시한다
모아달라고
닫고
리뉴얼
버려야
뒷받침할
주문한
내걸면서
대변되는
유래
유지된
추천해
예정이다네이버는
바깥
국가나
세계경제는
재외공관
축소를
같다며
시스템온칩
공약도
감소세는
모호한
증권업계에
들어줄
세금을
대작
비대면으로
대놓고
통화정책
이동형
조사된
침해와
됐으면
업종의
식사를
있다의
기사의
뒤따를
조사해
억만달러에서
상해
선발하는
있다국민은행
설비로
장비나
원가절감
분석도
연상시킨다는
초격차
체결해
폭풍은
코로나바이러스감염증코로나으로
내면서
아직은
시각이
달보다
기자이데일리
우타르프라데시주와
움직인
포함되어
구조조정
조건인
전세계에서
좋고
가구가
월평균
막바지에
내놓았다일
위치가
나섰고
셈이다분기
브랜드가
적용된다네이버는
전환한
머물렀던
불황으로
골라
전년억
누리집에
경제협력개발기구
소프트뱅크의
보안카드
항체약물
전세계로
강좌를
비밀번호
재개했다고
슬퍼요
억만장자
베스텍을
발동된
조정을
사망에
세대들의
벗어날
진단검사
해결하지
모두에
유리
확실하지
상승과
제공하려는
법인인
몰렸다
만들

명인
수출의
세부적인
년마다
희망퇴직
지급했다
불투명하다
꽉
상태이며
부적합한
주문하면
보너스
글로벌의
세계보건기구가
언급하기도
프로젝트가
시정률
주춤한
버리고
지구는
투싼
대기질이
참여정부
카드사와
답변이
타사
나타났지만
인상했다
기록이다
용도에
벤치마크
조치할
수출절벽을
시승을
단체를
깃든
결함내용을
꺼질
움직이는데
강도를
유동성을
힐링
해지
억원씩
인체에서
리스크를
간단하게
인천은
나라다
불가피하다며
하나금융투자
긴급히
도시에서는
만명에서
제작결함정보를
모집을
천억원에
웃돌았다
단속이
채우게
맞지만
급속한
툴에만
모잠비크
의견
느낌의
제정해
상승은
동남아에서
조조원의
했다현지
반짝
달했다고
됐다면서
생산량을
부담금을
가동되는
다수당을
성남
계획도
밝혀지면서
환자관리
만여명의
리콜에
일상에서
고급스러움을
선발에
하반기에
상태다봉쇄
산업별
동결
억천만
실행할
해소한다는
절실한
산소호흡기를
업체별
카카오에
불확실성으로
친숙하고
제공하면서
기기가
폐쇄했다
서울에서
하던
없다일
멈추는
팔을
총대를
애로사항이
나가고
억달러
변호사
내놓겠다는
세종뉴시스
꾸준하다
하락했다
명백한
개선돼
윙
체제로
투표자를
연장하면서
위배되는
국가적
높인다는
추정
곧바로
원하고
영양을
보인다권경원기자
항공기의
통상적인
걸림돌이
통과하면서
완화할
끝날
공언하면서
예상한다고
일련의
급변하면서
제한돼
충족하기
초까지
설치한다
진행해야
미팅은
이전보다는
추진해왔다
사각지대에
방해
난다
현재로서는
입체적으로
섰다
베이징
비중에
패밀리
경쟁력은
인적성
중학교가
걱정을
호가
사물을
보조금보다
유행
월말까지
여닫이
테스트하는
그려진
투표소를
성장했고
태도
참가했다
많았던
있다올해
안정적이고
덕에
짧게는
카드도
직원에
더하고
표현하며
검사가
마실
차들이
혁신방안
합병했다
공급도
중계할
총선거가
회복세는
신조
다가갈
당뇨병
가득하다
코스닥시장
국론을
언급하지
구글의
아닌데
자리잡으면서
변하지
금액과
증권업계
부문에서는
눈치를
구성한다
처음이다분기
발족한
맞섰다
거치며
예정한
소비를
모으기
폐에
수치로
받

In [21]:
duple_dict

[('있다', 2677),
 ('있는', 2472),
 ('년', 2099),
 ('코로나', 1765),
 ('위해', 1561),
 ('통해', 1546),
 ('것으로', 1490),
 ('재배포', 1370),
 ('대한', 1181),
 ('위한', 1096),
 ('대', 936),
 ('함께', 928),
 ('등을', 925),
 ('무단전재', 831),
 ('제공', 821),
 ('중', 813),
 ('도', 807),
 ('더', 800),
 ('국내', 792),
 ('오후', 769),
 ('이번', 741),
 ('서울', 710),
 ('같은', 704),
 ('따라', 699),
 ('다양한', 675),
 ('하는', 658),
 ('따르면', 629),
 ('가장', 626),
 ('지난해', 620),
 ('있도록', 602),
 ('많은', 567),
 ('있다고', 564),
 ('것이라고', 561),
 ('이후', 549),
 ('하고', 533),
 ('대해', 532),
 ('오는', 529),
 ('것이', 509),
 ('온라인', 500),
 ('때', 497),
 ('관계자는', 492),
 ('올해', 483),
 ('일부터', 480),
 ('또', 479),
 ('큰', 476),
 ('후', 472),
 ('가운데', 464),
 ('지난달', 458),
 ('모든', 450),
 ('모두', 449),
 ('이날', 446),
 ('두', 445),
 ('현재', 444),
 ('후보가', 442),
 ('것은', 440),
 ('글로벌', 426),
 ('최대', 424),
 ('등의', 420),
 ('새로운', 420),
 ('했다', 417),
 ('신종', 417),
 ('로', 416),
 ('코로나바이러스', 413),
 ('있다는', 413),
 ('밝혔다', 413),
 ('미국', 406),
 ('최근', 405),
 ('뒤', 402),
 ('없는', 402),
 ('세계', 

In [22]:
stopwords = ['에','는','은','을','했','에게',
             '있','이','의','하','한','다',
             '과','때문','할','수','무단',
             '따른','및','금지','전재',
             '경향신문','기자','는데','가',
             '등','들','파이낸셜','저작','등','뉴스']

leng = len(stopwords)
print(leng)
for duple in duple_dict:
    if not duple[0] in stopwords:
        stopwords.append(duple_dict[0])
        leng+=1
        if leng==100:
            break

31


In [23]:
print(len(stopwords))

100


In [24]:
text_data1 = preprocessing(df['news'],"Mecab",stopwords)
text_data2 = preprocessing(df['news'],"komoran",stopwords)

In [25]:
#Mecab
X_train, X_test, y_train, y_test = train_test_split(text_data1, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

# komoran
X_train, X_test, y_train, y_test = train_test_split(text_data2, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       IT/과학       0.76      0.80      0.78       224
          경제       0.73      0.90      0.81       218
          사회       0.87      0.77      0.82       234
       생활/문화       0.86      0.71      0.78       225

    accuracy                           0.80       901
   macro avg       0.80      0.80      0.80       901
weighted avg       0.81      0.80      0.80       901

              precision    recall  f1-score   support

       IT/과학       0.76      0.79      0.78       224
          경제       0.71      0.88      0.79       218
          사회       0.86      0.76      0.80       234
       생활/문화       0.83      0.71      0.76       225

    accuracy                           0.78       901
   macro avg       0.79      0.78      0.78       901
weighted avg       0.79      0.78      0.78       901



### 불용어 2

In [26]:
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스','노컷뉴스','매일신문','그','나오','가지','씨','시키','만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신','안','어떤','내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번','나','다른','어떻','여자','개','전','들','사실','이렇','점','싶','말','정도','좀','원','잘','통하','소리','놓','권','자','재','될','고','되','며','주','인','시','일','파','만','총','게','태','스','크','를','어','당','각','지난','관련','지만','사람','입니다','습니다','밝혔','지만','면서','지난','우리','이상','이용','경우','뉴시스','까지','다는','다면']

In [27]:
text_data1 = preprocessing(df['news'],"Mecab",stopwords)
text_data2 = preprocessing(df['news'],"komoran",stopwords)

#Mecab
X_train, X_test, y_train, y_test = train_test_split(text_data1, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

# komoran
X_train, X_test, y_train, y_test = train_test_split(text_data2, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       IT/과학       0.76      0.81      0.78       224
          경제       0.72      0.90      0.80       218
          사회       0.86      0.76      0.81       234
       생활/문화       0.86      0.69      0.77       225

    accuracy                           0.79       901
   macro avg       0.80      0.79      0.79       901
weighted avg       0.80      0.79      0.79       901

              precision    recall  f1-score   support

       IT/과학       0.76      0.79      0.78       224
          경제       0.71      0.89      0.79       218
          사회       0.86      0.75      0.80       234
       생활/문화       0.83      0.70      0.76       225

    accuracy                           0.78       901
   macro avg       0.79      0.78      0.78       901
weighted avg       0.79      0.78      0.78       901



### 불용어 3

In [28]:
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스','노컷뉴스','매일신문','그','나오','가지','씨','시키','만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신','안','어떤','내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번','나','다른','어떻','여자','개','전','들','사실','이렇','점','싶','말','정도','좀','원','잘','통하','소리','놓','권','자','재','될','고','되','며','주','인','시','일','파','만','총','게','태','스','크','를','어','당','각','지난','관련','지만','사람','입니다','습니다','밝혔','지만','면서','지난','우리','이상','이용','경우','뉴시스','까지','다는','다면']

In [29]:
text_data1 = preprocessing(df['news'],"Mecab",stopwords)
text_data2 = preprocessing(df['news'],"komoran",stopwords)

#Mecab
X_train, X_test, y_train, y_test = train_test_split(text_data1, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

# komoran
X_train, X_test, y_train, y_test = train_test_split(text_data2, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       IT/과학       0.76      0.81      0.78       224
          경제       0.72      0.90      0.80       218
          사회       0.86      0.76      0.81       234
       생활/문화       0.86      0.69      0.77       225

    accuracy                           0.79       901
   macro avg       0.80      0.79      0.79       901
weighted avg       0.80      0.79      0.79       901

              precision    recall  f1-score   support

       IT/과학       0.76      0.79      0.78       224
          경제       0.71      0.89      0.79       218
          사회       0.86      0.75      0.80       234
       생활/문화       0.83      0.70      0.76       225

    accuracy                           0.78       901
   macro avg       0.79      0.78      0.78       901
weighted avg       0.79      0.78      0.78       901



In [30]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('clf', SGDClassifier(loss='hinge', penalty='l2',
                       alpha=1e-3, random_state=42,
                       max_iter=5, tol=None))
])
text_clf.fit(X_train_tfidf, y_train)
y_pred = text_clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.79      0.81      0.80       224
          경제       0.80      0.86      0.83       218
          사회       0.86      0.81      0.83       234
       생활/문화       0.81      0.77      0.79       225

    accuracy                           0.81       901
   macro avg       0.81      0.81      0.81       901
weighted avg       0.81      0.81      0.81       901



In [31]:
# 데이터 불균형한 상태로 하는 경우
df=pd.concat([df1,df2],axis=0,ignore_index=True)
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스','노컷뉴스','매일신문','그','나오','가지','씨','시키','만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신','안','어떤','내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번','나','다른','어떻','여자','개','전','들','사실','이렇','점','싶','말','정도','좀','원','잘','통하','소리','놓','권','자','재','될','고','되','며','주','인','시','일','파','만','총','게','태','스','크','를','어','당','각','지난','관련','지만','사람','입니다','습니다','밝혔','지만','면서','지난','우리','이상','이용','경우','뉴시스','까지','다는','다면']

text_data = preprocessing(df['news'],"Mecab",stopwords)

X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)



text_clf = Pipeline([('clf', SGDClassifier(loss='hinge', penalty='l2',
                       alpha=1e-3, random_state=42,
                       max_iter=5, tol=None))
])
text_clf.fit(X_train_tfidf, y_train)
y_pred = text_clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       IT/과학       0.88      0.89      0.88       506
          경제       0.83      0.66      0.73       247
          사회       0.82      0.90      0.85       760
       생활/문화       0.86      0.82      0.84       694

    accuracy                           0.84      2207
   macro avg       0.84      0.81      0.83      2207
weighted avg       0.84      0.84      0.84      2207



데이터 자체가 불균형 하고 중복되는게 많아서 그런 것 같다.
최대한 중복을 없애고 데이터를 더 추가한다면 정확도가 많이 향상될 것으로 보인다.
SGD가 짱이다.